In [1]:
%pip install langchain==0.0.319
%pip install openai==0.28.1
%pip install chromadb==0.4.14
%pip install sentence-transformers
%pip install pypdf pdfplumber
%restart_python

  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached sqlalchemy-2.0.45-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (9.5 kB)
  Using cached aiohttp-3.13.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (8.1 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-

UsageError: Line magic function `%restart_python` not found.


In [10]:
import os
import shutil
import hashlib
import pdfplumber

from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# DeepSeek API (or OpenAI-compatible)
os.environ["OPENAI_API_KEY"] = "sk-aa47d49919ad4a8795605774abad2b49"
os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com/v1"

# Vectorstore output directory
persist_dir = "./chroma_db_1222_5"

# PDF files to process
pdf_files = ["Jockey202425.pdf", "HKJC_140A.pdf", "HKJC_Annual Report 2425 part1.pdf", "HKJC_Annual Report 2425 part2.pdf"]

# CELL 3: Processing and Embedding
all_chunks = []
table_count = 0
seen_hashes = set()

# Smart text splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)

for pdf_path in pdf_files:
    # 1. Extract text using PyPDFLoader
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    for doc in docs:
        doc.metadata["source"] = pdf_path
        doc.metadata["fallback_prompt"] = "You are a PDF assistant. If data is unclear or missing, say 'Not enough information.'"
        doc.metadata["type"] = "text"

    # 2. Split into chunks & deduplicate
    text_chunks = splitter.split_documents(docs)
    for chunk in text_chunks:
        content_hash = hashlib.md5(chunk.page_content.strip().encode()).hexdigest()
        if content_hash not in seen_hashes:
            all_chunks.append(chunk)
            seen_hashes.add(content_hash)

    # 3. Extract tables via pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        prev_table = None
        prev_header = None
        prev_page = None
        prev_start_page = None

        for i, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            for table in tables:
                clean_table = [
                    [str(cell) if cell is not None else "" for cell in row]
                    for row in table if any(cell is not None for cell in row)
                ]
                if not clean_table or len(clean_table) < 2:
                    continue

                current_header = clean_table[0]
                current_content = clean_table[1:]

                if (
                    prev_table is not None and
                    len(prev_header) == len(current_header) and
                    current_header == prev_header
                ):
                    prev_table.extend(current_content)
                    prev_page = i + 1
                else:
                    if prev_table:
                        table_count += 1
                        table_str = "\n".join(["\t".join(row) for row in [prev_header] + prev_table])
                        table_doc = Document(
                            page_content=table_str,
                            metadata={
                                "source": pdf_path,
                                "type": "table",
                                "table_id": f"{pdf_path}_table_{table_count}",
                                "columns": ", ".join(prev_header),
                                "page_start": prev_start_page,
                                "page_end": prev_page,
                                "fallback_prompt": "You are a PDF assistant. The following is a table. Read rows carefully."
                            }
                        )
                        all_chunks.append(table_doc)

                    prev_table = current_content
                    prev_header = current_header
                    prev_start_page = i + 1
                    prev_page = i + 1

        # Final table at EOF
        if prev_table:
            table_count += 1
            table_str = "\n".join(["\t".join(row) for row in [prev_header] + prev_table])
            table_doc = Document(
                page_content=table_str,
                metadata={
                    "source": pdf_path,
                    "type": "table",
                    "table_id": f"{pdf_path}_table_{table_count}",
                    "columns": ", ".join(prev_header),
                    "page_start": prev_start_page,
                    "page_end": prev_page,
                    "fallback_prompt": "You are a PDF assistant. The following is a table. Read rows carefully."
                }
            )
            all_chunks.append(table_doc)

# 4. Delete old ChromaDB if exists (important after downgrade)
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

# 5. Embed and persist with explicit CPU device specification
if len(all_chunks) == 0:
    print("❌ No chunks to embed! Verify PDF content or OCR requirements.")
else:
    embedding = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cpu"}, encode_kwargs={"normalize_embeddings": True}  # Explicitly set CPU device
    )
    
    vectordb = Chroma.from_documents(
        all_chunks,
        embedding=embedding,
        persist_directory=persist_dir
    )
    vectordb.persist()

    print(f"✅ Embeddings created and saved to: `{persist_dir}`")
    print(f"🔍 Total chunks embedded: {len(all_chunks)} (including {table_count} merged tables)")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ Embeddings created and saved to: `./chroma_db_1222_5`
🔍 Total chunks embedded: 1001 (including 71 merged tables)


In [ ]:
%pip install --upgrade langchain==0.1.16 langchain-community==0.0.33 langchain-openai==0.0.8


In [7]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("HKJC_140A.pdf")
docs = loader.load()
print(f"📄 Loaded {len(docs)} pages")
print("📝 Sample text:", docs[0].page_content[:300])
# ✅ Vectorstore path (same as you used)
persist_dir = "./chroma_db_1222_5"

# ✅ Load the same embedding model used to build the DB
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# ✅ Load the vectorstore from disk
vectordb = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# ✅ Check how many documents are in the store
print("📄 Number of documents in vectorstore:", vectordb._collection.count())

# ✅ Ask a test question
user_query = "who are An An and Ke Ke?"
results = vectordb.similarity_search(user_query, k=3)

# ✅ Print results
if results:
    for i, doc in enumerate(results, 1):
        print(f"\n🔹 Result {i}:\n{doc.page_content[:500]}")  # truncate output
else:
    print("❌ No relevant documents found.")

for i, doc in enumerate(docs[:3]):
    print(f"\nPage {i+1} content length: {len(doc.page_content)}")
    print(doc.page_content[:300])

📄 Loaded 118 pages
📝 Sample text: 


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


📄 Number of documents in vectorstore: 1001

🔹 Result 1:
3
Board of Stewards
董事局
Mr Michael T H Lee 
JP 
(Chairman)
利子厚先生(主席)
The Hon Martin Liao 
GBM GBS JP 
(Deputy Chairman)
廖長江先生(副主席)
Dr Silas S S Yang 
JP
楊紹信博士
Miss Anita Fung Yuen Mei 
BBS JP
馮婉眉小姐
Mr Philip K W Lo
羅啟華先生
Mr Andrew W B R Weir 
BBS MBE JP
韋安祖先生
Mr Lester G Huang 
SBS JP
黃嘉純先生
Dr Henry H L Chan
陳衍里醫生
The Hon Bernard Charnwut Chan 
GBM GBS JP
陳智思先生
Mr Jackson Woo Ka Biu
胡家驃先生
Mrs Ann Kung Yeung Yun Chi 
BBS JP
龔楊恩慈女士
Mr Nicholas D Hunsworth
孔思和先生

🔹 Result 2:

SECTOR

🔹 Result 3:
	BEAUTY JOY
美麗同享
A S Cruz
告東尼
Eleanor Kwok Law Kwai
Chun & Patrick Kwok Ho
Chuen
郭羅桂珍與郭浩泉	

Page 1 content length: 0


Page 2 content length: 0


Page 3 content length: 0

